In [1]:
import json
import numpy as np
import scipy.stats
import pandas as pd
import seaborn as sns
from statannot import add_stat_annotation
import matplotlib.pyplot as plt
import scipy.stats as stats

# Use the RT in the model_based_var
model_based_var =json.load(open('../model_fitting/model_based_var/model_based_generated_var_RL_performance_avg_aff_score_precise_rt.json', 'r'))



In [2]:
model_based_var[0].keys()

dict_keys(['workerID', 'shownCond', 'shownStim', 'pWinOfMov', 'trial_index', 'response_list', 'reward_list', 'state_list', 'affordance_list', 'is_new_block', 'out_list', 'rt', 'gen_params', 'chosen_prob', 'chosen_q', 'all_q', 'chosen_aff', 'all_aff', 'wc_list', 'xi_aff_list', 'xi_q_list', 'rpe_list', 'pppe_aff_list', 'pppe_q_list'])

In [8]:
def getStay(shownCond, response):
    n0 = 0
    n1 = 0
    n2 = 0
    n3 = 0
    l0 = []
    l1 = []
    l2 = []
    l3 = []
    idx = []
    stay_list = []
    for i, cond in enumerate(shownCond):
        if cond == "congruent high":
            idx.append(n0)
            l0.append(response[i])
            n0 += 1
        elif cond == "incongruent high":
            idx.append(n1)
            l1.append(response[i])
            n1 += 1            
        elif cond == "congruent low":
            idx.append(n2)
            l2.append(response[i])
            n2 += 1   
        elif cond == "incongruent low":
            idx.append(n3)
            l3.append(response[i])
            n3 += 1
    for i, resp in enumerate(response):
        cond = shownCond[i]
        tI = idx[i]
        if cond == "congruent high":
            l = l0
        elif cond == "incongruent high":
            l = l1
        elif cond == "congruent low":
            l = l2
        elif cond == "incongruent low":
            l = l3
        try:
            next_resp = l[tI+1]
            stay = int(next_resp == resp)
        except:
            stay = -1
        stay_list.append(stay)
                
    return stay_list

def shownCond2trialByCond(shownCond, rewarded):
    n0 = 0
    n1 = 0
    n2 = 0
    n3 = 0
    idx = []
    for i, cond in enumerate(shownCond):
        if cond == "congruent high":
            idx.append(n0)
            if rewarded[i]:
                n0 += 1
        elif cond == "incongruent high":
            idx.append(n1)
            if rewarded[i]:
                n1 += 1            
        elif cond == "congruent low":
            idx.append(n2)
            if rewarded[i]:
                n2 += 1   
        elif cond == "incongruent low":
            idx.append(n3)
            if rewarded[i]:
                n3 += 1
    return idx

def shown_conds2code(shown_conds):
    out_cong = []
    out_hl = []
    for cond in shown_conds:
        if cond == "congruent high":
            out_cong.append(1)
            out_hl.append(1)
        elif cond == "incongruent high":
            out_cong.append(0)
            out_hl.append(1)
        elif cond == "congruent low":
            out_cong.append(1)
            out_hl.append(0)
        else: #"incongruent low"
            out_cong.append(0)
            out_hl.append(0)
    return out_cong, out_hl

def stim2code(stim):
    if "Pinch" in stim:
        return 0
    elif "Clench" in stim:
        return 1
    elif "Poke" in stim:
        return 2
    else:
        return 3

def code2action(code):
    if code == 0:
        return "pinch"
    elif code == 1:
        return "clench"
    elif code == 2:
        return "poke"
    else:
        return "palm"
    
def stim_array2code(stim_array):
    stim_list = stim_array
    code_list = [stim2code(stim) for stim in stim_list]
    return np.array(code_list)

def idx21hot(idx):
    hot = np.zeros((idx.size, idx.max()+1))
    hot[np.arange(idx.size), idx] = 1
    return (hot[:,:-1]).tolist()

def prob2correct(prob):
    out = []
    for p in prob:
        out.append(p.index(max(p)))
    return np.array(out)

subj_id_list = []
mov_list = []
is_affordance_list = []
is_correct_list = []
cong_list = []
hl_list = []
trial_index_list = []
rewarded_trial_index_list = []
rt_list = []
log_rt_list = []
rewarded_list = []
stay_list = []
wc_list = []
chosen_prob_list = []
chosen_q_list = []
chosen_aff_list = []
uncertainty_list = []

for i in range(len(model_based_var)):
    correct = prob2correct(model_based_var[i]['pWinOfMov'])
    subj_id_list += [i]*len(model_based_var[i]['shownStim'])
    shownStim = stim_array2code(model_based_var[i]['shownStim'])
    selectedMov = [int(x)-1 for x in model_based_var[i]['response_list']]
    mov_list +=  [code2action(x) for x in selectedMov]
    is_affordance_list += [float(x) for x in (shownStim == selectedMov)]
    is_correct_list += [float(x) for x in (correct == selectedMov)]
    cong, hl = shown_conds2code(model_based_var[i]['shownCond'])
    cong_list += cong
    hl_list += hl
    trial_index_list += [float(x) for x in model_based_var[i]['trial_index']]
    rewarded_trial_index_list += shownCond2trialByCond(model_based_var[i]['shownCond'], model_based_var[i]['reward_list'])
    rewarded_list += model_based_var[i]['reward_list']
    log_rt_list += [np.log(max(1000*x, 1)) for x in model_based_var[i]['rt']]
    rt_list += [1000*x for x in model_based_var[i]['rt']]
    stay_list += getStay(model_based_var[i]['shownCond'], selectedMov)
    wc_list += model_based_var[i]['wc_list']
    chosen_prob_list += model_based_var[i]['chosen_prob']
    chosen_q_list += model_based_var[i]['chosen_q']
    beta = np.exp(model_based_var[i]['gen_params'][1])
    beta2 = np.exp(model_based_var[i]['gen_params'][2])
    beta6 = model_based_var[i]['gen_params'][5]
    beta7 = model_based_var[i]['gen_params'][6]
    beta8 = model_based_var[i]['gen_params'][7]
    aff_p = [p[:3] for p in model_based_var[i]['all_aff']]
    q_p = [p[:3] for p in model_based_var[i]['all_q']]
    wc = model_based_var[i]['wc_list']
    p_all = [np.array(aff_p[j])*wc[j] + np.array(q_p[j])*(1-wc[j]) for j in range(len(aff_p))]
    var = ([np.array([x*(1-x) for x in p]) for p in model_based_var[i]['pWinOfMov']])
    #uncertainty_list += [np.sum(p_all[j]*var[j]) for j in range(len(p_all))]
    uncertainty_list += [var[j][selectedMov[j]] for j in range(len(var))]
    chosen_aff_list += [beta2*a for a in model_based_var[i]['chosen_aff']]
    
data = {'subj':subj_id_list, 'action':mov_list, 'choosing_aff': is_affordance_list,
       'choosing_correct': is_correct_list, 'cong_incong': cong_list, 'rewarded': rewarded_list,
       'high_low': hl_list, 'trial_idx': trial_index_list, 'rewarded_trial_idx':rewarded_trial_index_list, 'stay': stay_list, 'rt': rt_list,
        'log_rt': log_rt_list, 'wc':wc_list, 'chosen_p': chosen_prob_list, 'chosen_q':chosen_q_list, 'chosen_aff': chosen_aff_list,
       'uncertainty':uncertainty_list}

df = pd.DataFrame(data)


In [9]:
#df.to_csv('/Users/YiSangHyun/Downloads/RT_behav.csv', index=False)
df

,subj,action,choosing_aff,choosing_correct,cong_incong,rewarded,high_low,trial_idx,rewarded_trial_idx,stay,rt,log_rt,wc,chosen_p,chosen_q,chosen_aff,uncertainty
0,0,poke,1.0,0.0,0,0,1,0.0,0,0,1512.611736,7.321593,0.865614,0.342269,0.000000,0.936429,0.16
1,0,poke,0.0,0.0,0,0,0,0.0,0,0,867.273327,6.765354,0.865614,0.301782,0.000000,0.381754,0.09
2,0,poke,0.0,0.0,1,1,0,0.0,0,1,914.339257,6.818202,0.865614,0.293002,0.000000,0.137321,0.09
3,0,poke,1.0,1.0,1,1,1,0.0,0,0,1097.738714,7.001008,0.865614,0.344821,0.000000,0.953750,0.16
4,0,pinch,1.0,0.0,0,0,0,1.0,0,0,1495.665382,7.310326,0.865614,0.414204,0.000000,0.754912,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14330,29,clench,1.0,1.0,1,1,0,17.0,36,1,751.392125,6.621928,0.980640,0.985632,0.308456,0.930877,0.24
14331,29,pinch,1.0,0.0,0,1,1,18.0,71,-1,787.864846,6.669327,0.980810,0.989545,0.394792,0.655000,0.16
14332,29,pinch,1.0,1.0,1,1,1,18.0,82,-1,748.827027,6.618508,0.875289,0.946492,0.800453,0.942222,0.16
14333,29,clench,0.0,1.0,0,0,0,18.0,33,-1,830.363610,6.721864,0.980809,0.981543,0.299001,0.344912,0.24


In [10]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.mixedlm('log_rt ~ (choosing_aff + choosing_correct + trial_idx + cong_incong + high_low + C(action))', df, groups=data["subj"],
                   re_formula = '~ (choosing_aff + choosing_correct + trial_idx + cong_incong + high_low + C(action))')
free = sm.regression.mixed_linear_model.MixedLMParams.from_components(
    np.ones(8), np.eye(8)
)
results = model.fit(method=["powell"], free=free)

print(results.summary())




/Users/YiSangHyun/anaconda3/envs/py3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                        Mixed Linear Model Regression Results
Model:                      MixedLM           Dependent Variable:           log_rt    
No. Observations:           14335             Method:                       REML      
No. Groups:                 30                Scale:                        0.1371    
Min. group size:            461               Log-Likelihood:               -6301.8381
Max. group size:            480               Converged:                    Yes       
Mean group size:            477.8                                                     
--------------------------------------------------------------------------------------
                                           Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------------
Intercept                                   6.915    0.051 135.275 0.000  6.815  7.015
C(action)[T.pinch]                         -0.006    0.010  -0.593 0

In [11]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
# cong = 1, incong = 0

df_ = df[(df['cong_incong']==0)]

model = smf.mixedlm('log_rt ~ (choosing_aff + choosing_correct + trial_idx + high_low + C(action))', df_, groups=df_["subj"],
                   re_formula = '~ (choosing_aff + choosing_correct + trial_idx + high_low + C(action))')
#model = smf.ols(formula='rt ~ (choosing_aff + choosing_correct + trial_idx + cong_incong + high_low + C(action))*C(subj)', data=df)
free = sm.regression.mixed_linear_model.MixedLMParams.from_components(
    np.ones(7), np.eye(7)
)
results = model.fit(method=["powell"], free=free)

print(results.summary())




/Users/YiSangHyun/anaconda3/envs/py3/lib/python3.7/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


                        Mixed Linear Model Regression Results
Model:                      MixedLM           Dependent Variable:           log_rt    
No. Observations:           7171              Method:                       REML      
No. Groups:                 30                Scale:                        0.1462    
Min. group size:            231               Log-Likelihood:               -3432.1411
Max. group size:            240               Converged:                    Yes       
Mean group size:            239.0                                                     
--------------------------------------------------------------------------------------
                                           Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------------
Intercept                                   6.938    0.053 130.726 0.000  6.834  7.042
C(action)[T.pinch]                         -0.032    0.014  -2.327 0